# Crawling data from 愛買
This notebook is part of code in project: Categorize product
Because of lacking enough data, I decide to crawl down some more data from other hypermarket, so this notebook is mainly about crawling data from 愛買

In [25]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests 

In [26]:
love_buy ="https://tw.mall.yahoo.com/store/%E6%84%9B%E8%B2%B7%E7%B7%9A%E4%B8%8A%E8%B3%BC%E7%89%A9:amart"
html = requests.get(love_buy).content

In [27]:
soup = BeautifulSoup(html , 'lxml')
#print(soup.prettify())
catagory = soup.find_all('div',id="mystoremenubar")

#catagory

In [28]:
href = []
for i in range(len(catagory)):
  small_catagory = catagory[i].find_all("li", class_ = "topmenu")
  #print(small_catagory)
  for j in range(len(small_catagory)):
    href.append(small_catagory[j].find("a")["href"])
    word = []

In [29]:
href

['https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=8&path=8',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=961&path=961',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=650&path=650',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=1018&path=1018',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=21&path=21',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=107&path=107',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=10&path=10',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=32&path=32',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=1800&path=1800',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=195&path=195',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=110&path=110',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=1801&path=1801',
 'https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=2099&path=2099',
 'https://tw.mall.yahoo.com/search?m=lis

In [30]:
def list_to_string(list_):
  return '~~'.join(list_)


In [31]:
def get_title(soup):
  title = soup.find("title").text
  word.append(title)
  #print(title)
  return title

In [32]:
def get_deep_data(href):
  print("get_deep_data")
  word = []
  para = requests.get(href).content
  soup = BeautifulSoup(para , 'lxml')
  name = get_title(soup)
  context = soup.find_all("li", class_ = "cat-breadcrumb")
  for i in range(len(context)):
    word.append((context[i].text.strip()))
    #print(context[i].text)
  word.append(name)
  print(list_to_string(word))
  return list_to_string(word)

In [33]:
site = "https://tw.mall.yahoo.com/search"

In [34]:
def get_down(all_text):
  #print("get down")
  keyword_list = ['下']
  if any(word in all_text for word in keyword_list):
    return True
  else:
    return False

In [35]:
def get_next_page(soup):
  #print("get next page")
  next = soup.find_all("div", id = "ypsaupg")
  for i in range(len(next)):
    next_ = next[i].find("div", class_ ="bd")
  #print(next_.text)
  if get_down(next_.text):
    page = next_.find_all("a")[-1]
    print(site + page["href"])
    return page["href"]
  else:
    return 0

In [36]:
para = requests.get("https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=8&s=-sc_salerank&b=816&view=image&path=8").content
soup = BeautifulSoup(para , 'lxml')
print(site + str(get_next_page(soup)))

https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=8&s=-sc_salerank&b=864&view=image&path=8
https://tw.mall.yahoo.com/search?m=list&sid=amart&ccatid=8&s=-sc_salerank&b=864&view=image&path=8


In [37]:
data = []

In [38]:
def get_data(href):
  data = []
  small_href = []
  para = requests.get(href).content
  soup = BeautifulSoup(para , 'lxml')
  #print(soup)
  context = soup.find_all("p", class_ = "desc")
  for i in range(len(context)):
    small_href.append((context[i].find("a")["href"]))
  if get_next_page(soup) != 0:
    data = data + (get_data(site + get_next_page(soup)))

  for i in range(len(small_href)):
    data.append(get_deep_data(small_href[i]))
  return data


In [39]:
len(href)

14

In [ ]:

from google.colab import files
whole_data = []
#href
for i in range(12,len(href)):
  print(i)
  whole_data = get_data(href[i])
  a = pd.DataFrame(whole_data)
  a.to_csv("love_buy_data.csv", encoding = "utf-8-sig")
  files.download('love_buy_data.csv')



串流輸出內容已截斷至最後 5000 行。
get_deep_data
超級商城~~美妝/藥妝/髮品~~美髮染護/沐浴美體~~身體清潔~~香皂/精油皂~~可瑞絲精油香皂Q10玫瑰100g【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~美食/纖體/保健~~飲料/食材/雜貨~~罐頭食品/調理包~~調理包~~樂雅樂洋蔥雞肉咖哩200G【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~美食/纖體/保健~~美顏美體/營養保健~~營養補給~~葡萄糖胺~~柏之暢二型膠原蛋白膠囊60顆【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~美妝/藥妝/髮品~~美髮染護/沐浴美體~~身體清潔~~沐浴乳/露~~EAU耀香水沐浴乳1000ml-湛藍微風【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~居家/寢具/家具~~五金/照明工具~~五金/修繕~~五金用品~~電池~~永備 碳鋅電池2號 4入/組【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~美食/纖體/保健~~飲料/食材/雜貨~~油品/調味品~~調味醬料~~蕃茄醬/辣椒醬/豆瓣醬~~萬家香海味山珍醬320G【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~美妝/藥妝/髮品~~美髮染護/沐浴美體~~臉部清潔~~洗臉~~洗面乳(膠/露)~~萊雅沙漠礦泥零油光控油洗面乳100ml【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~居家/寢具/家具~~五金/照明工具~~五金/修繕~~五金用品~~掛勾~~DIY居家原木U型掛勾214-3A(中)【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~美食/纖體/保健~~飲料/食材/雜貨~~泡麵/麵條/速食粥~~速食粥~~GREENSTAN台式紅豆紫米粥40g X15【愛買】 | 愛買線上購物 - Yahoo奇摩超級商城
get_deep_data
超級商城~~美食/纖體/保健~~小吃熟食/糕餅/零食~~餅乾/零嘴/點心/糖果~~其他零嘴/點心/

In [ ]:

a.to_csv("love_buy_data.csv", encoding = "utf-8-sig")
files.download('love_buy_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data

In [ ]:
test = pd.DataFrame(whole_data)

In [ ]:
whole_data[-2]

In [ ]:
new_data = []
for i in range(len(whole_data)):
  new_data.append(str(whole_data[i]).split(" "))


In [ ]:
test = pd.DataFrame(new_data)

In [ ]:
test

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,水產海鮮,魚,台灣現流極鮮鯛魚片1片(110g/片)【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None,None
1,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,冷凍冷藏食品,包子/饅頭/米漢堡,饅頭,義美銀絲捲420g/包【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None
2,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,生鮮肉品,牛肉,美國特選級冷凍霜降牛排1包(500G/包)【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None,None
3,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,水產海鮮,螺/蚌/蛤/蚵/貝,日本特大扇貝(5~6粒)(300G±5%/包)【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None,None
4,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,冷凍冷藏食品,微波食品/調理包,調理包,元進莊嚴選組(油雞腿)【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,生鮮肉品,牛肉,美國特選冷凍無骨牛小排300G/盒X3【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None,None
74,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,冷凍冷藏食品,火鍋料/炸物,火鍋料,桂冠蝦餃100g【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None
75,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,冷凍冷藏食品,水餃/餛飩/鍋貼,水餃,冰冰好料理高麗菜豬肉熟水餃935g【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None
76,超級商城,美食/纖體/保健,冷凍/生鮮/甜點,冷凍冷藏食品,披薩/餅皮/餡餅,蛋餅/抓餅/蔥油餅,金品鮮採青蔥手工烙餅5片/包(約600g)【愛買冷凍】,|,愛買線上購物,-,Yahoo奇摩超級商城,None,None


In [ ]:
from google.colab import files
test.to_csv('love_buy_test.csv',  encoding = "utf-8-sig") 
files.download('love_buy_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>